In [54]:
import os
import filecmp
import hashlib

class BackupInspectorClass:
    def __init__(self):
        pass

    def compare_directories(self, refrence_directory, target_directory):
        comparison = filecmp.dircmp(refrence_directory, target_directory, ignore=None)

        self.left_only_total += len(comparison.left_only)
        self.right_only_total += len(comparison.right_only)
        self.diff_files_total += len(comparison.diff_files)
        self.same_files_total += len(comparison.same_files)

        for filename in comparison.left_only:
            missing_files_list.append(os.path.join(refrence_directory, filename))        # must be faster way to tdo this than looping through as it is just taking  alsit to another list and appending constant string to begibg of each entry in list, similarly below

        if self.hash_choice != "None":                                    # PROBABLOY QUICKER TO REMOVE THIS AND JUST DIRECTLY DO HASH CHECK AT HIS POINT RATHER THAN AFTER ALL DIRECTORIES HAVE BEEN COMPARED
            for filename in comparison.same_files:  
                self.possible_match_list_refrence.append(os.path.join(refrence_directory,filename))            #append filenames of all same_files_total to possible_match_list for checksum scanning
                self.possible_match_list_target.append(os.path.join(target_directory,filename))              #append filenames of all same_files_total to possible_match_list for checksum scanning

        # Recursively compare subdirectories
        for subdirname in comparison.common_dirs:
            self.compare_directories(os.path.join(refrence_directory, subdirname), os.path.join(target_directory, subdirname))

        return left_only_total, right_only_total, diff_files_total, same_files_total

    def hash(self, file_path):

        # Initialize MD5 or SHA-1 hash object.
        if self.hash_choice == "Md5":
            hash_operator = hashlib.md5()
        elif self.hash_choice == "Sha1":
            hash_operator = hashlib.sha1()
        
        # Calculate hashes for the files
            with open(file_path, 'rb') as file:
                while True:
                    data = file.read(8192)  # Read data in chunks.
                    if not data:
                        break
                    hash_operator.update(data)

        return hash_operator.hexdigest()

    def checksum_comparison(self):
        # Iterate through the list of possible matches.
        for reference_path, target_path in zip(possible_match_list_refrence, possible_match_list_target):
            
            reference_hash = hash(reference_path)
            target_hash = hash(target_path)
            
            # If hashes DO NOT match
            if reference_hash != target_hash:
                print(f"Files don't match: {reference_path} and {target_path}")
                # Add the reference path to the list of missing files.
                missing_files_list.append(reference_path)
                # add one to the count of diff files and one to the left only total
                self.diff_files_total += 1
                self.left_only_total += 1

                # remove the reference and target  paths from thier respective lists of possible matches
                possible_match_list_refrence.remove(reference_path)
                possible_match_list_target.remove(target_path)
                # remove one from the count of same files
                self.same_files_total -= 1

    def run(self, reference_dirs, target_dirs, hash_choice):
        self.hash_choice = hash_choice
        self.left_only_total = 0
        self.right_only_total = 0
        self.diff_files_total = 0
        self.same_files_total = 0
        self.possible_match_list_refrence = []
        self.possible_match_list_target = []
        self.missing_files_list = []

        for reference_directory, target_directory in zip(reference_dirs, target_dirs):
            self.compare_directories(reference_directory, target_directory)

        if hash_choice != "None":
            self.checksum_comparison()

        # Calculate the total number of files in the reference and target directories.
        total_files_reference = self.left_only_total + self.same_files_total
        total_files_target = self.right_only_total + self.same_files_total

        # Calculate the percentage of files missing from the target directory.
        percentage_missing = round((self.left_only_total / total_files_reference) * 100, 2)

        # Calculate the percentage of files that are the same between the reference and target directories.
        percentage_same = round((self.same_files_total / total_files_reference) * 100, 2)

        return self.left_only_total, self.right_only_total, self.diff_files_total, self.same_files_total, self.missing_files_list, total_files_reference, total_files_target, percentage_missing, percentage_same
    

# Example usage:
reference_directory = [r"A:\Users\Ada\Desktop", r"A:\Users\Ada\Desktop"]
target_directory = [r"A:\Users\Ada\Desktop", r"A:\Users\Ada\Desktop"]
hash_choice = "Md5"

backup_inspector = BackupInspectorClass()
left_only_total, right_only_total, diff_files_total, same_files_total, missing_files_list, total_files_reference, total_files_target, percentage_missing, percentage_same = backup_inspector.run(reference_directory, target_directory, hash_choice)

print(f"total_files_reference: {total_files_reference}")
print(f"total_files_target: {total_files_target}")
print(f"left_only_total: {left_only_total}")
print(f"right_only_total: {right_only_total}")
print(f"diff_files_total: {diff_files_total}")
print(f"same_files_total: {same_files_total}")
print(f"percentage_same: {percentage_same}")
print(f"percentage_missing: {percentage_missing}")
print(f"missing_files_list: {missing_files_list}")


total_files_reference: 14030
total_files_target: 14030
left_only_total: 0
right_only_total: 0
diff_files_total: 0
same_files_total: 14030
percentage_same: 100.0
percentage_missing: 0.0
missing_files_list: []


In [51]:
import timeit

def run_backup_inspector(hash_choice):
    backup_inspector = BackupInspectorClass()
    left_only_total, right_only_total, diff_files_total, same_files_total, missing_files_list = backup_inspector.run(reference_directory, target_directory, hash_choice)

# Define the reference_directory and target_directory variables

hash_choices = ["None", "Md5", "SHA-1"]

for hash_choice in hash_choices:
    stmt = f"run_backup_inspector('{hash_choice}')"
    setup = "from __main__ import run_backup_inspector"
    
    # Time the execution of the code
    execution_time = timeit.timeit(stmt, setup, number=100)
    
    print(f"Hash Choice: {hash_choice}")
    print(f"Execution Time: {execution_time} seconds")
    print()


Hash Choice: None
Execution Time: 238.9262598999776 seconds

Hash Choice: Md5
Execution Time: 251.22141569998348 seconds

Hash Choice: SHA-1
Execution Time: 258.2752649000031 seconds

